<a href="https://colab.research.google.com/github/Alberto-Moura/Market-Basket-Analysis/blob/main/marketBasketAnalysis_OTP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Market basket analysis (MBA)**

técnica de análise de dados usada para identificar padrões de compra em conjunto. O objetivo é descobrir quais produtos são comprados juntos com mais frequência, ajudando na tomada de decisões como:

🔹 Cross-selling → Sugestão de produtos complementares (ex: "Clientes que compraram camiseta também compraram meias").<br>
🔹 Layout de lojas → Posicionar produtos próximos para aumentar vendas.<br>
🔹 Promoções → Criar combos estratégicos com base nos itens frequentemente comprados juntos.<br>

A análise é baseada em três métricas principais:

🔹Suporte → Frequência do conjunto de itens na base de compras.<br>
🔹Confiança → Probabilidade de comprar um item, dado que outro foi comprado.<br>
🔹Lift → Mede se a compra conjunta acontece por acaso ou há uma forte relação
entre os itens.<br>

(Análise de cesta de mercado) - Loja Franquia

**Importando as bibliotecas necessárias**

In [ ]:
# Importando as bibliotecas necessárias
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

**Leitura dos dados**

In [ ]:
# Carregue o arquivo Excel. Verifique o caminho do arquivo conforme necessário.
df = pd.read_excel("dados_loja.xlsm", sheet_name='dados_loja')

In [ ]:
# Visualize as primeiras linhas para confirmar a estrutura dos dados
print("Dados originais:")
df.head()

**Agrupamento dos produtos por transação**

In [ ]:
df['data_str'] = df['DATA EMISSÃO'].dt.strftime('%d%m%Y')
df['NF_str'] = df['nº NF'].astype(str)
df['cupom_id'] = df['NF_str'] + df['data_str']

In [ ]:
# Supondo que cada transação seja identificada pelo 'cupom_id'
# Cria uma lista de transações onde cada transação é uma lista de produtos comprados juntos.
transactions = df.groupby('cupom_id')['MODELO'].apply(list).tolist()

In [ ]:
# Exibe alguns exemplos de transações
print("\nExemplo de transações:")
for i, trans in enumerate(transactions[:5]):
    print(f"Transação {i+1}: {trans}")

**Transformação dos dados para o formato one-hot encoding**

In [ ]:
# O TransactionEncoder converte a lista de transações em um array binário, onde cada coluna representa um produto.
te = TransactionEncoder()
te_array = te.fit_transform(transactions)
df_trans = pd.DataFrame(te_array, columns=te.columns_)

In [ ]:
# Visualize o DataFrame transformado
print("\nDados após one-hot encoding:")
df_trans.head()

**Aplicação do algoritmo Apriori para encontrar itemsets frequentes**

In [ ]:
# Defina um suporte mínimo (exemplo: 2% das transações). Ajuste conforme a necessidade.
min_support = 0.02
frequent_itemsets = apriori(df_trans, min_support=min_support, use_colnames=True)

In [ ]:
# Ordene os itemsets por suporte para ver os mais frequentes
frequent_itemsets = frequent_itemsets.sort_values(by='support', ascending=False)
print("\nItemsets frequentes (suporte >= {:.0%}):".format(min_support))
frequent_itemsets.head()

**Geração das regras de associação**

In [ ]:
# Defina uma confiança mínima (exemplo: 30%). Você pode ajustar conforme seu interesse.
min_confidence = 0.3
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=min_confidence)

In [ ]:
# Ordene as regras pela confiança para ver as mais fortes
rules = rules.sort_values(by='confidence', ascending=False)
print("\nRegras de associação (confiança >= {:.0%}):".format(min_confidence))
rules.head()

**Exemplo de interpretação dos resultados:**

In [ ]:
# Construindo a tabela de probabilidade de compra (confiança)
# Para cada par de produtos (A, B), vamos calcular:
#   confiança(A -> B) = suporte(A e B) / suporte(A)

# Obtemos a lista de produtos a partir das colunas do DataFrame one-hot encoded
produtos = df_trans.columns

# Calcula o suporte individual de cada produto (média da coluna True/False)
suportes = df_trans.mean()

# Cria um DataFrame vazio para armazenar as confianças, indexado e com colunas iguais à lista de produtos
conf_table = pd.DataFrame(index=produtos, columns=produtos)

# Para cada par de produtos (A, B), calcular confiança(A -> B)
for produto_a in produtos:
    for produto_b in produtos:
        # Se A e B forem iguais, não faz sentido calcular (poderíamos colocar 1 ou deixar vazio)
        if produto_a == produto_b:
            conf_table.loc[produto_a, produto_b] = None
        else:
            # Suporte da conjunção: transações onde A e B aparecem juntas
            suporte_a_b = (df_trans[produto_a] & df_trans[produto_b]).mean()
            # Se o suporte de A for zero, a confiança não pode ser calculada; caso contrário, calcula
            if suportes[produto_a] > 0:
                conf = suporte_a_b / suportes[produto_a]
            else:
                conf = 0
            conf_table.loc[produto_a, produto_b] = conf

# Converter os valores para float (exceto os nulos)
conf_table = conf_table.astype(float)

print("\nTabela de probabilidade de compra (confiança):")
conf_table